In [162]:
import cv2
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage.transform import resize
from tensorflow.python.keras.layers import Conv3D, ReLU, MaxPooling3D, Reshape, LSTM, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.python.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.optimizer_v2.gradient_descent import SGD
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [163]:
classes = ["NonViolence", "Violence"]

## Listing all class videos with labels

In [164]:
label = 0
file_list = []
Y = []
for name in classes:
    file_names = glob.glob(f"dataset/Real Life Violence Dataset/{name}/*.*")
    file_list.extend(file_names)

    Y.extend([label for _ in range(len(file_names))])
    label += 1

Y = np.array(Y)

In [219]:
class Frames:
    fps = 5

    @classmethod
    def read_fr(cls, arr):
        """
        method for reading frames from videos
        :return: array of video frames
        """
        videos = []
        for index, video in enumerate(arr):
            print("Completed", round(index / len(arr) * 100, 2), end="\r")
            resized_frames = []

            caption = cv2.VideoCapture(video)
            frame_rate = caption.get(Frames.fps)

            while caption.isOpened():
                frame_id = caption.get(1)
                ret, frame = caption.read()

                if not ret:
                    break

                if frame_id % np.floor(frame_rate) == 0:
                    resized_frames.append(cv2.resize(frame, (64, 64)))

            videos.append(resized_frames)
            caption.release()

        return videos

    @classmethod
    def select_fr(cls, arr):
        """
        method for selecting frames
        """
        videos = []
        for i in range(len(arr)):
            frames = []
            for linear_sep in np.linspace(0, len(arr[i]) - 1, num=Frames.fps):
                frames.append(arr[i][int(linear_sep)])

            if len(frames) <= 5:
                videos.append(frames)

        np_videos = np.array(videos)
        return np_videos

In [166]:
X_train = Frames.select_fr(Frames.read_fr(file_list))

In [167]:
X = X_train.copy()

## Splitting data into train and test

In [168]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=42, shuffle=True)

## Creating neural network

In [170]:
model = tf.keras.models.Sequential()
model.add(BatchNormalization(input_shape=(5,64,64,3)))
model.add(Conv3D(filters=64, kernel_size=3,padding='same'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPooling3D(pool_size=(1,2,2)))

model.add(Conv3D(filters=64, kernel_size=3,padding='same'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPooling3D(pool_size=(1,2,2)))

model.add(Conv3D(filters=128,kernel_size=3,padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPooling3D(pool_size=(1,2,2)))

model.add(Conv3D(filters=128,kernel_size=3,padding='same'))
model.add(BatchNormalization())
model.add(ReLU())

model.add(Reshape((5,128*64)))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32, return_sequences=False,dropout=0.5))

model.add(Dense(512,'relu'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(0.5))

model.add(Dense(128,'relu'))

model.add(Dense(1, 'sigmoid'))

model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),metrics=['accuracy'])
model.build(input_shape=(None, 5,64,64,3))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_90 (Bat  (None, 5, 64, 64, 3)     12        
 chNormalization)                                                
                                                                 
 module_wrapper_19 (ModuleWr  (None, 5, 64, 64, 64)    5248      
 apper)                                                          
                                                                 
 batch_normalization_91 (Bat  (None, 5, 64, 64, 64)    256       
 chNormalization)                                                
                                                                 
 module_wrapper_20 (ModuleWr  (None, 5, 64, 64, 64)    0         
 apper)                                                          
                                                                 
 module_wrapper_21 (ModuleWr  (None, 5, 32, 32, 64)   

In [171]:
model.fit(X_train, y_train, batch_size=10, epochs=7, validation_data=(X_test, y_test))

Epoch 1/7
160/160 [==============================] - 586s 4s/step - loss: 0.6925 - accuracy: 0.5556 - val_loss: 0.6859 - val_accuracy: 0.6325
Epoch 2/7
160/160 [==============================] - 568s 4s/step - loss: 0.5710 - accuracy: 0.7088 - val_loss: 0.6241 - val_accuracy: 0.6400
Epoch 3/7
160/160 [==============================] - 569s 4s/step - loss: 0.4750 - accuracy: 0.7719 - val_loss: 0.5329 - val_accuracy: 0.7325
Epoch 4/7
160/160 [==============================] - 563s 4s/step - loss: 0.4254 - accuracy: 0.8069 - val_loss: 0.4289 - val_accuracy: 0.8000
Epoch 5/7
160/160 [==============================] - 565s 4s/step - loss: 0.3726 - accuracy: 0.8413 - val_loss: 0.4211 - val_accuracy: 0.8000
Epoch 6/7
160/160 [==============================] - 565s 4s/step - loss: 0.3499 - accuracy: 0.8388 - val_loss: 0.4078 - val_accuracy: 0.8150
Epoch 7/7
160/160 [==============================] - 567s 4s/step - loss: 0.2962 - accuracy: 0.8825 - val_loss: 0.3789 - val_accuracy: 0.8300


In [188]:
tf.keras.models.save_model(model, "saved_model_v1")

INFO:tensorflow:Assets written to: saved_model_v1\assets


INFO:tensorflow:Assets written to: saved_model_v1\assets


In [193]:
result = model.evaluate(X_test, y_test, batch_size=10)

40/40 [==============================] - 17s 430ms/step - loss: 0.3789 - accuracy: 0.8300


In [174]:
loss, accuracy = result
print(f"Accuracy: {round(accuracy * 100, 3)} %")

Accuracy: 83.0 %


## Tests
If we get the 0 as prediction value that means there is a violence in the video, otherwise not.

In [197]:
preds = model.predict(X_test)

In [198]:
y_preds = []
for pred in preds:
    if pred[0] * 100 > 50:
        y_preds.append(1)
    else:
        y_preds.append(0)

In [199]:
report = classification_report(y_test, y_preds)

In [200]:
def classification_report_to_dataframe(str_representation_of_report):
    split_string = [x.split(' ') for x in str_representation_of_report.split('\n')]
    column_names = ['']+[x for x in split_string[0] if x!='']
    values = []
    for table_row in split_string[1:-1]:
        table_row = [value for value in table_row if value!='']
        if table_row:
            values.append(table_row)
    for i in values:
        for j in range(len(i)):
            if i[1] == 'avg':
                i[0:2] = [' '.join(i[0:2])]
            if len(i) == 3:
                i.insert(1,np.nan)
                i.insert(2, np.nan)
            else:
                pass
    report_to_df = pd.DataFrame(data=values, columns=column_names)
    return report_to_df

In [201]:
classification_report_to_dataframe(report)

,,precision,recall,f1-score,support
0,0,0.84,0.81,0.83,199
1,1,0.82,0.85,0.83,201
2,accuracy,NaN,NaN,0.83,400
3,macro avg,0.83,0.83,0.83,400
4,weighted avg,0.83,0.83,0.83,400


In [202]:
from PIL import Image
Image.fromarray(X_test[0][0], "RGB").save("test.jpg")

In [203]:
confusion_matrix(y_test, y_preds)

array([[161,  38],
       [ 30, 171]], dtype=int64)

## Loading saved model

In [195]:
model = tf.keras.models.load_model("saved_model_v1")

In [196]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_90 (Bat  (None, 5, 64, 64, 3)     12        
 chNormalization)                                                
                                                                 
 module_wrapper_19 (ModuleWr  (None, 5, 64, 64, 64)    0         
 apper)                                                          
                                                                 
 batch_normalization_91 (Bat  (None, 5, 64, 64, 64)    256       
 chNormalization)                                                
                                                                 
 module_wrapper_20 (ModuleWr  (None, 5, 64, 64, 64)    0         
 apper)                                                          
                                                                 
 module_wrapper_21 (ModuleWr  (None, 5, 32, 32, 64)   

In [205]:
X_train

array([[[[[ 77,  65,  72],
          [ 71,  65,  69],
          [ 81,  66,  71],
          ...,
          [181, 182, 184],
          [ 69,  57,  59],
          [ 89,  70,  72]],

         [[ 78,  70,  78],
          [ 88,  70,  67],
          [ 94,  72,  60],
          ...,
          [202, 198, 201],
          [ 70,  55,  58],
          [ 92,  73,  74]],

         [[ 83,  66,  75],
          [164, 138,  59],
          [209, 169,  44],
          ...,
          [ 59,  48,  52],
          [ 70,  53,  56],
          [ 96,  74,  76]],

         ...,

         [[  4,   9,  14],
          [  7,   5,  10],
          [ 12,   7,  13],
          ...,
          [ 38,  28,  28],
          [ 39,  29,  29],
          [ 39,  29,  29]],

         [[  4,  10,  14],
          [ 11,   9,  14],
          [ 15,  10,  16],
          ...,
          [ 39,  29,  29],
          [ 40,  30,  30],
          [ 40,  30,  30]],

         [[  4,  10,  14],
          [ 11,   9,  14],
          [ 18,  13,  19],
         

In [206]:
array = np.load("nv_video.npy")

In [207]:
array.ndim

5

In [209]:
model.predict(np.expand_dims(X_test[0], axis=0))

array([[0.5213612]], dtype=float32)

In [212]:
y_test[0]

1

In [211]:
preds

array([[5.2136111e-01],
       [5.2767992e-04],
       [8.5176551e-01],
       [6.7782420e-01],
       [8.9387405e-01],
       [9.8893452e-01],
       [4.9538910e-02],
       [7.0356357e-01],
       [2.5361884e-01],
       [8.2208520e-01],
       [2.1850467e-03],
       [9.5641279e-01],
       [9.6933651e-01],
       [9.0250373e-03],
       [1.5757084e-03],
       [6.9975555e-03],
       [5.7096970e-01],
       [2.0311791e-01],
       [9.9261737e-01],
       [8.9381504e-01],
       [1.4891624e-03],
       [9.1715527e-01],
       [5.0342268e-01],
       [9.7950017e-01],
       [7.0776373e-02],
       [1.8049031e-02],
       [9.6764493e-01],
       [3.0882248e-01],
       [5.1174873e-01],
       [7.2030127e-03],
       [1.8558204e-03],
       [7.7749944e-01],
       [2.5102794e-03],
       [2.8557479e-02],
       [8.3459187e-01],
       [2.6377946e-02],
       [3.7703457e-01],
       [1.7341900e-01],
       [9.2982006e-01],
       [4.0328532e-01],
       [6.3481897e-02],
       [8.882746

In [213]:
fra = Frames()

In [230]:
x = fra.select_fr(fra.read_fr(file_list[1773:1774]))

In [231]:
file_list[1773:1774]

['dataset/Real Life Violence Dataset/Violence\\V_795.mp4']

In [232]:
model.predict(x)

array([[0.8451073]], dtype=float32)

In [237]:
a = fra.read_fr(["test_video_nv.mp4"])

In [238]:
model.predict(fra.select_fr(a))

array([[0.3363331]], dtype=float32)